In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Perceptron, RidgeCV, Lasso
from sklearn.decomposition import PCA

from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_selection import VarianceThreshold
#from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import torch
from torch import optim
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F 


In [ ]:
train_X = pd.read_csv("/kaggle/input/lish-moa/train_features.csv")
train_y = pd.read_csv("/kaggle/input/lish-moa/train_targets_scored.csv")
train_y_ns = pd.read_csv("/kaggle/input/lish-moa/train_targets_nonscored.csv")
#train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, test_size=0.3)

test_X = pd.read_csv("/kaggle/input/lish-moa/test_features.csv")
print(test_X.shape)

In [ ]:
train_X.head(2)

In [ ]:
train_ctl_vehicle_idx = train_X[train_X['cp_type']=='ctl_vehicle'].index
test_ctl_vehicle_idx = test_X[test_X['cp_type'] == 'ctl_vehicle'].index

train_X = train_X.drop(train_ctl_vehicle_idx).reset_index(drop=True)
train_y = train_y.drop(train_ctl_vehicle_idx).reset_index(drop=True)
test_X = test_X.drop(test_ctl_vehicle_idx).reset_index(drop=True)

train_X, test_X = train_X.drop('cp_type', axis=1), test_X.drop('cp_type', axis=1)

In [ ]:
X_columns, y_columns = train_X.columns, train_y.columns
X_columns


In [ ]:
pd.set_option('display.max_columns', 200)
train_y.head(2)
print(train_y.shape)

In [ ]:
print("Train features shape:", train_X.shape, "Train Scored Targets:", train_y.shape)
#print("Validation features shape:{}", val_X.shape, "Validation Scored Targers:", val_y.shape)

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(14,4))
#sns.countplot(train_X['cp_type'], ax=axes[0])
sns.countplot(train_X['cp_dose'], ax=axes[0])
sns.countplot(train_X['cp_time'], ax=axes[1])

In [ ]:
g_features = [col for col in train_X.columns if col.startswith('g-')]
c_features = [col for col in train_X.columns if col.startswith('c-')]

In [ ]:
print("Number of gene features:{}, number of cell features:{}".format(len(g_features), len(c_features)))

In [ ]:
n_c_features = len(c_features)
step = n_c_features // 9

fig, axes = plt.subplots(nrows=3,ncols=3, figsize=(10, 10))

for  i in range(0, 9, 1):
    row, col = i // 3, i % 3
    features = train_X.loc[:, 'c-{}'.format(i * step)]
    sns.kdeplot(features, ax=axes[row, col])

In [ ]:
n_g_features = len(g_features)
step = n_g_features // 9

fig, axes = plt.subplots(nrows=3,ncols=3, figsize=(10, 10))

for  i in range(0, 9, 1):
    row, col = i // 3, i % 3
    features = train_X.loc[:, 'g-{}'.format(i * step)]
    sns.kdeplot(features, ax=axes[row, col])

In [ ]:
def plot_std(train_X, columns):
    df = train_X[columns]
    stds = df.std(axis=0)
    sns.distplot(stds)
    plt,show()


def plot_correlation(df, columns):

    corr = df[columns].corr()
    print(corr)
    mask = np.triu(np.ones_like(corr, dtype=bool))
    f, ax = plt.subplots(figsize=(11, 9))
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
    plt.show()
    

def number_of_activations(train_y):
    
    df = train_y.drop(['sig_id'], axis=1).astype(bool).sum(axis=1).reset_index()

    df.columns = ['row', 'count']
    df = data.groupby(['count'])['row'].count().reset_index()

    fig = sns.barplot(data, y=data['row'],  x="count", 
        title='Number of activations in targets for every sample')

    fig.show()

def find_top_correlation(train_x, train_y, x_columns, target):
    
    target_features = train_y[target]
    correlations = []
    for x_column in x_columns:
        features = train_x[x_column]
        corr_coef = np.corrcoef(features, target_features)
        correlations.append((x_column, corr_coef))
    
    top5 = sorted(target_features, key=lambda x:x[1], inverse=True)
    return top5

In [ ]:
def drop_cp(df):
    return df[df['cp_type'] == 'trt_cp'].reset_index(drop=True)


In [ ]:
#ind_tr = train_X[train_X['cp_type'] == 'ctl_vehicle'].index
#ind_te = test_X[test_X['cp_type'] == 'ctl_vehicle'].index

test_ids = test_X.sig_id

In [ ]:
#print(val_X)
#train_y.iloc[list(ind_tr)].sum(axis=1)#.head(2)

In [ ]:
#most_popular_occurancies
#print(train_y.iloc[0])
scored = train_y.sum(axis=0)[1:].sort_values(ascending=False) #[1:] so as sort without id colums
#print(scored)
#print(scored)
scored[:20].plot(kind='barh', fontsize=14, figsize=(5,20))

In [ ]:
print(train_X.shape)
for  i in range(0, len(g_features), len(g_features)//10):
    sns.kdeplot(train_X.loc[:, g_features[i]])
    plt.show()

In [ ]:
c_features = [col for col in train_X.columns if col.startswith('c-')]
print(train_X.shape)
for  i in range(0, len(c_features), len(c_features)//10):
    sns.kdeplot(train_X.loc[:, c_features[i]])
    plt.show()

In [ ]:
def normal_split(train_X, train_y):
    X_n_columns = len(train_X.columns)
    y_n_columns = len(train_y.columns)
    #print(train_X.shape, train_y.shape)
    
    train_X_y = pd.merge(train_X, train_y ,on='sig_id')
    #print("After merge:", train_X_y.shape)
    #train_X_y = train_X_y.drop(['sig_id', 'cp_type'], axis=1)
    
    #print("After drop:", train_X_y.shape)
    
    train_X = train_X_y.iloc[:, :X_n_columns]
    train_y = train_X_y.iloc[:, X_n_columns:]
    return train_X, train_y

train_X, train_y = normal_split(train_X, train_y)
#val_X, val_y = normal_split(val_X, val_y)
train_y.shape
#train_X.head(3)

In [ ]:
#-----------------------------Preprocession block-------------------------------
def cat_dfs(dataFrames):
    sizes = [len(df) for df in dataFrames]
    
    concatenated_df = pd.concat(dataFrames, axis=0).reset_index(drop=True)
    
    return concatenated_df, sizes

def drop_high_correlated(dataFrames, threshold=0.9):
    df, sizes = cat_dfs(dataFrames)
    cols = df.columns
    above_90 = []

    for i in range(0, len(cols)):
        for j in range(i+1, len(cols)):
            #print(i,j)
            if abs(df[cols[i]].corr(df[cols[j]])) > threshold:
                above_90.append(cols[i])
                
    print("number high correlated:{}".format(len(above_90))) 
    
    df = df.drop(above_90, axis=1)
    return __split_data_frame(df, sizes)


def __split_data_frame(df, sizes):
    
    last_size = 0
    splitted_dfs = []
    
    for size in sizes:
        tmp_df = df[last_size:size + last_size]
        splitted_dfs.append(tmp_df)
        last_size = size + last_size
    
    return splitted_dfs


def map_categorical(dataFrames):
    concatenated_df, sizes = cat_dfs(dataFrames)
    #concatenated_df['cp_type'] = concatenated_df['cp_type'].map({'trt_cp':0, 'ctl_vehicle':1})
    concatenated_df['cp_time'] = concatenated_df['cp_time'].map({24:0, 48:1, 72:2})
    concatenated_df['cp_dose'] = concatenated_df['cp_dose'].map({'D1':0, 'D2':1})
    
    return __split_data_frame(concatenated_df, sizes)
    

def filter_variance(dataFrames, threshold=0.5):
    concatenated_df, sizes = cat_dfs(dataFrames)
    print(sizes)
    VT = VarianceThreshold(threshold)
    
    
    variances = VT.fit(concatenated_df).variances_
    
    columns_names = concatenated_df.columns[variances < threshold]
    print(columns_names)
    concatenated_df = concatenated_df.drop(columns_names, axis=1)
    
    return __split_data_frame(concatenated_df, sizes)


    
def add_PCA(dataFrames, g_n_components, c_n_components):
    sizes = [len(df) for df in dataFrames]
    
    concatenated_df, sizes = cat_dfs(dataFrames)
    
    g_features = [col for col in concatenated_df.columns if col.startswith('g-')]
    c_features = [col for col in concatenated_df.columns if col.startswith('c-')]
    

    pca_transformer_g = PCA(n_components=g_n_components)
    pca_transformer_c = PCA(n_components=c_n_components)

    pca_features_g = pd.DataFrame(pca_transformer_g.fit_transform(concatenated_df[g_features]),
                                  columns=["PCA_g_{}".format(i) for i in range(g_n_components)])
    pca_features_c = pd.DataFrame(pca_transformer_c.fit_transform(concatenated_df[c_features]),
                                 columns=["PCA_c_{}".format(i) for i in range(c_n_components)])
        
    
    

    new_df = pd.concat([concatenated_df[g_features], pca_features_g,
                        concatenated_df[c_features], pca_features_c], axis=1)

        
    return __split_data_frame(new_df, sizes)


def add_features(dataFrames):
    
    df, sizes = cat_dfs(dataFrames)
    g_features = [col for col in df.columns if col.startswith('g-')]
    c_features = [col for col in df.columns if col.startswith('c-')]
    
    df['g_mean'] = df[g_features].mean(axis=1)
    df['g_std'] = df[g_features].std(axis=1)
    df['g_sum'] = df[g_features].sum(axis=1)
    
    df['c_mean'] = df[c_features].mean(axis=1)
    df['c_std'] = df[c_features].std(axis=1)
    df['c_sum'] = df[c_features].sum(axis=1)
    df = pd.concat([df, g_mean, g_std, g_sum, c_mean, c_std, c_sum], axis=1)
    
    return __split_data_frame(df, sizes)


def add_new(dataFrames):
    df, sizes = cat_dfs(dataFrames)
    
    #df['time_dose'] = df['cp_time'].astype(str)+df['cp_dose']
    features_g = [col for col in df.columns if col.startswith('g-')]
    features_c = [col for col in df.columns if col.startswith('c-')]
    
    df['g_sum'] = df[features_g].sum(axis = 1)
    df['g_mean'] = df[features_g].mean(axis = 1)
    df['g_std'] = df[features_g].std(axis = 1)
    df['g_kurt'] = df[features_g].kurtosis(axis = 1)
    df['g_skew'] = df[features_g].skew(axis = 1)
    df['c_sum'] = df[features_c].sum(axis = 1)
    df['c_mean'] = df[features_c].mean(axis = 1)
    df['c_std'] = df[features_c].std(axis = 1)
    df['c_kurt'] = df[features_c].kurtosis(axis = 1)
    df['c_skew'] = df[features_c].skew(axis = 1)
    df['gc_sum'] = df[features_g + features_c].sum(axis = 1)
    df['gc_mean'] = df[features_g + features_c].mean(axis = 1)
    df['gc_std'] = df[features_g + features_c].std(axis = 1)
    df['gc_kurt'] = df[features_g + features_c].kurtosis(axis = 1)
    df['gc_skew'] = df[features_g + features_c].skew(axis = 1)
    
    
    for feature in features_c:
        df[f'{feature}_squared'] = df[feature] ** 2
    
    return __split_data_frame(df, sizes)

In [ ]:
def preprocess(dataFrames, cat_features, con_features):
    
    new_dfs = []
    dataFrames_cat = [ df[cat_features] for df in dataFrames]
    dataFrames_con = [ df[con_features] for df in dataFrames]
    
    dataFrames_con = filter_variance(dataFrames_con)
    print("Variance filtered")
    #dataFrames_con = drop_high_correlated(dataFrames_con)
    print("High correlated features dropped")
    dataFrames_con = add_PCA(dataFrames_con, 30, 10)
    print("PCA added")
    dataFrames_con = add_new(dataFrames_con)
    print("Additional featires added")
    dataFrames_cat = map_categorical(dataFrames_cat)
    print("Categorical features transformed")
    for df_cat, df_con in zip(dataFrames_cat, dataFrames_con):
        new_dfs.append(pd.concat((df_cat, df_con), axis=1))
    
    return new_dfs



In [ ]:


train_X = train_X.drop('sig_id', axis=1, errors='ignore')
test_X = test_X.drop('sig_id', axis=1, errors='ignore')

cat_features = ['cp_time', 'cp_dose']
con_features = [colname for colname in train_X.columns 
                if colname not in cat_features]


train_X, test_X = preprocess([train_X, test_X], cat_features, con_features)

print("train_X shape:{}, test_X:{}".format(train_X.shape, test_X.shape))

In [ ]:
train_X.head(2)

In [ ]:
'''
folds_train_X = train_X.copy()
folds_train_y = train_y.copy()

Fold = KFold(n_splits=6, shuffle=True, random_state=42)
for n, (train_index, val_index) in enumerate(Fold.split(folds_train_X, folds_train_y)):
    folds_train_X.loc[val_index, 'fold'] = int(n)
    folds_train_y.loc[val_index, 'fold'] = int(n)
#folds['fold'] = folds['fold'].astype(int)
folds_train_X['fold'] = folds_train_X['fold'].astype(int)
folds_train_y['fold'] = folds_train_y['fold'].astype(int)
print(folds_train_X.shape, folds_train_y.shape)
'''

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class MoADataset(Dataset):
    def __init__(self, X, y=None):
        if y is not None:
            assert X.shape[0] == y.shape[0]
        
        self.X = torch.tensor(X.to_numpy(), dtype=torch.float)
        self.y = None
        if y is not None:
            self.y = torch.tensor(y.to_numpy(), dtype=torch.float)
        
    def __getitem__(self, idx):

        tensor_X = self.X[idx]
        if self.y is not None:
            tensor_y = self.y[idx]
            return tensor_X, tensor_y
        else:
            return tensor_X
    
    def __len__(self):
        return len(self.X)

In [ ]:
dataset = MoADataset(train_X, train_y)
train_dataloader = DataLoader(dataset, batch_size=10)

In [ ]:
class MoANet(nn.Module):
    
    def __init__(self, input_size, output_size):
        super().__init__()
        
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = 256

        self.wn = nn.utils.weight_norm
        self.mlp = nn.Sequential(
                          self.wn(nn.Linear(input_size, self.hidden_size)),
                            
                          #nn.BatchNorm1d(self.hidden_size),
                          nn.Dropout(0.3),
                          nn.ELU(),
                          self.wn(nn.Linear(self.hidden_size, self.hidden_size)),
                          #nn.BatchNorm1d(self.hidden_size),
                          nn.Dropout(0.3),
                          nn.ELU(),
                          self.wn(nn.Linear(self.hidden_size, self.output_size)))
        
        
    def forward(self, x):
        x = self.mlp(x)
        return x

net = MoANet(train_X.shape[1], train_y.shape[1])

In [ ]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.2)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.2)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        #x = self.batch_norm1(x)
        #x = self.dropout1(x)
        x = F.leaky_relu(self.dense1(x), 1e-3)
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x =F.leaky_relu(self.dense2(x), 1e-3)
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

In [ ]:
#train_X, test_X = process_data([train_X, test_X])


In [ ]:
# train, validate and test on train data and test data
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for X, y in dataloader:
        optimizer.zero_grad()
        X, y = X.to(device), y.to(device)
        #print(inputs.shape)
        outputs = model(X)
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        outputs = model(X)
        loss = loss_fn(outputs, y)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for X in dataloader:
        X = X.to(device)

        with torch.no_grad():
            outputs = model(X)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds
   


In [ ]:
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 26
BATCH_SIZE = 256
LEARNING_RATE = 6e-4
WEIGHT_DECAY = 1e-5
NFOLDS = 7
EARLY_STOPPING_STEPS = 10
EARLY_STOP = True
HIDDEN_SIZE=2048

In [ ]:
def run_training(fold_number, train_X, train_y, test_X, train_indexes, val_indexes, seed):
    #seed_everything(seed)
    
    x_train = train_X.iloc[train_indexes].reset_index(drop=True)
    y_train = train_y.iloc[train_indexes].reset_index(drop=True)
    
    x_val = train_X.iloc[val_indexes].reset_index(drop=True)
    y_val = train_y.iloc[val_indexes].reset_index(drop=True)
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_val, y_val)
    
    trainloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    model = Model(
        num_features=train_X.shape[-1],
        num_targets=train_y.shape[-1],
        hidden_size=HIDDEN_SIZE,
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e1, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((train_X.shape[0], train_y.shape[-1]))
    best_loss = np.inf
    
    print(f"FOLD: {fold_number}")
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_fn, trainloader, DEVICE)
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        print(f"EPOCH: {epoch}, train_loss: {train_loss}, valid_loss: {valid_loss}")
        
        if valid_loss < best_loss:
            early_step =0
            best_loss = valid_loss
            oof[val_indexes] = valid_preds
            torch.save(model.state_dict(), f"FOLD{fold_number}_.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    
    #--------------------- PREDICTION---------------------
    testdataset = MoADataset(test_X)
    testloader = DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=train_X.shape[-1],
        num_targets=train_y.shape[-1],
        hidden_size=HIDDEN_SIZE,
    )
    
    model.load_state_dict(torch.load(f"FOLD{fold_number}_.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_X), train_y.shape[-1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions

In [ ]:
def run_k_fold(NFOLDS, train_X, train_y, test_X, seed):
    oof = np.zeros((train_y.shape))
    predictions = np.zeros((test_X.shape[0], train_y.shape[-1]))
    kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=seed)
    
    for fold, (train_indexes, val_indexes)  in enumerate(kf.split(train_X)):
        oof_, pred_ = run_training(fold, train_X, train_y, test_X, train_indexes, val_indexes, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_ 
        
    return oof, predictions

In [ ]:
SEED = [1973, 1998]
oof = np.zeros((train_X.shape[0], train_y.shape[-1]))
predictions = np.zeros((test_X.shape[0], train_y.shape[-1]))

for seed in SEED:
    print(f"SEED: {seed}")
    
    oof_, predictions_ = run_k_fold(NFOLDS, train_X, train_y, test_X, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

#train[target_cols] = oof
#test[target_cols] = predictions


In [ ]:
train_prediction_df= pd.DataFrame(data=oof, columns=train_y.columns)
test_prediction_df = pd.DataFrame(data=predictions, columns=train_y.columns)

In [ ]:
train_ctl_vehicle_idx
test_ctl_vehicle_idx

tts = pd.read_csv("/kaggle/input/lish-moa/train_targets_scored.csv")
train = pd.read_csv("/kaggle/input/lish-moa/train_features.csv")
tstf = pd.read_csv("/kaggle/input/lish-moa/test_features.csv")
print(tts.shape)
#print(len(train_ctl_vehicle_idx))
dropped_sig_id_train = pd.DataFrame(tts.iloc[:,0].drop(train_ctl_vehicle_idx, axis=0).reset_index(drop=True), columns=['sig_id']) 
#print(dropped_sig_id_train.shape, train_prediction_df.shape)

train_prediction_df = pd.concat([dropped_sig_id_train, train_prediction_df], axis=1)
valid_results = tts.drop(columns=train_y.columns).merge(train_prediction_df, on='sig_id', how='left').fillna(0)


y_true = tts[train_y.columns].values
y_pred = valid_results[train_y.columns].values


score = 0
for i in range(y_true.shape[1]):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ 
score /= train_y.shape[1]
    
print("CV log_loss: ", score)
#print(*(zip(valid_results.sig_id, tts.sig_id)))
#dropped_sig_id_test = tstf.iloc[test_ctl_vehicle_idx]['sig_id']
#test_prediction_df = pd.concat([dropped_sig_id_test, test_prediction_df])
#test_results = tstf.drop(columns=train.iloc[:, 1:].columns).merge(test_prediction_df, on='sig_id', how='left').fillna(0)

#print("CV log_loss:{}".format(log_loss(tts.iloc[:,1:], valid_results.iloc[:, 1:])))




In [ ]:
dropped_sig_id_test = pd.DataFrame(tstf.iloc[:,0].drop(test_ctl_vehicle_idx, axis=0).reset_index(drop=True), columns=['sig_id']) 
#print(dropped_sig_id_train.shape, train_prediction_df.shape)

test_prediction_df = pd.concat([dropped_sig_id_test, test_prediction_df], axis=1)
test_results = tstf.drop(columns=tstf.columns[1:]).merge(test_prediction_df, on='sig_id', how='left').fillna(0)
test_results.shape

In [ ]:
test_results.to_csv("submission.csv", index=None)

In [ ]:
#preds = predict(net, val_dataloader)

In [ ]:
#print(val_X)
#val_log_loss = evaluate(preds, val_y.to_numpy())
#print("You get such quality of log_loss:{}".format(val_log_loss))

In [ ]:
#preds = predict(net, test_dataloader)

In [ ]:
'''
preds = pd.DataFrame(preds, columns=train_y.columns)
preds['sig_id'] = test_ids
cols = preds.columns.tolist()
cols = cols[-1:] + cols[:-1]
preds = preds[cols]
preds.set_index(keys='sig_id', drop=False)
preds.to_csv("submission.csv", index=None)
'''

In [ ]:
#readed = pd.read_csv("submission.csv")
#readed.shape

In [ ]:
#readed.head(2)

In [ ]:
import lightgbm as lgb

def one_to_all_reg(reg_model, train_X, train_y):
    models = {}
    #print(train_y)
    for idx, target_name in enumerate(train_y.columns):
        print("Train #:{}. {}".format(idx, target_name))
        targets = train_y[target_name]
        #print(targets)
        reg_model.fit(train_X, targets)
        models[target_name] = reg_model
        
    return models

def predict_ensemble(test_X, models, targets_names):
    df_prediction = pd.DataFrame()
    for target_name in targets_names:
        df_prediction[target_name] = models[target_name].predict(test_X)
        
    return df_prediction

def create_equal_ordering(pred_df, true_df):
    assert pred_df.shape == true_df.shape
    
    numpy_pred = np.zeros(pred_df.shape)
    numpy_true = np.zeros(true_df.shape)
    
    for idx, target_name in enumerate(true_df.columns):
        numpy_pred[:, idx] = pred_df[target_name].to_numpy()
        numpy_true[:, idx] = true_df[target_name].to_numpy()
        
    return numpy_pred, numpy_true

In [ ]:
from  sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.svm import SVC
'''
params = {'num_leaves': 490,
          'min_child_weight': 0.03,
          'feature_fraction': 0.55,
          'bagging_fraction': 0.9,
          'min_data_in_leaf': 150,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.01,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'binary_logloss',
          "verbosity": 0,
          'reg_alpha': 0.4,
          'reg_lambda': 0.6,
          'random_state': 47
         }

reg_model = lgb.LGBMClassifier(params)

models = one_to_all_reg(reg_model, train_X, train_y)
'''

In [ ]:
#models

In [ ]:
#preds = predict_ensemble(val_X, models, val_y.columns)#\
#numpy_pred, numpy_true = create_equal_ordering(preds, val_y)
#loss = evaluate(numpy_pred, numpy_true)
#print("Your ensemble have log score:{}".format(loss))